In [7]:
import zipfile
import pandas as pd
import os

# Deschide fisierul zip
with zipfile.ZipFile("C:\\Users\\Iuliana\\Desktop\\medium-articles-master\\natural-language-processing\\topic-modeling\\data\\NIPS Papers.zip", "r") as zip_ref:
    zip_ref.extractall("temp")

papers = pd.read_csv("temp/NIPS Papers/papers.csv")


In [8]:
# elimina coloanele
papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)


In [9]:
# Load the regular expression library
import re

# elimina punctuatia
papers['paper_text_processed'] = \
papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Uppercase devin lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())


In [10]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

# se elimina stop words
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Iuliana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import gensim.corpora as corpora

# Se creaza dictionar
id2word = corpora.Dictionary(data_words)
texts = data_words

# Se foloseste conceptul de bags of words astfel documentul este transfomat in cuvinte separate necontand ordinea
corpus = [id2word.doc2bow(text) for text in texts]

In [12]:
from pprint import pprint

num_topics = 10

# Build LDA model
LDAmodel = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word,num_topics=num_topics)

# Se printeaza 10 cuvinte cheie pentru temele alese
pprint(LDAmodel.print_topics())
doc_lda = LDAmodel[corpus]

[(0,
  '0.006*"algorithm" + 0.006*"learning" + 0.005*"training" + 0.004*"time" + '
  '0.004*"set" + 0.004*"data" + 0.004*"network" + 0.004*"one" + 0.003*"model" '
  '+ 0.003*"using"'),
 (1,
  '0.006*"data" + 0.006*"model" + 0.005*"function" + 0.005*"set" + '
  '0.004*"time" + 0.004*"two" + 0.004*"one" + 0.003*"learning" + '
  '0.003*"training" + 0.003*"neural"'),
 (2,
  '0.007*"model" + 0.005*"set" + 0.005*"learning" + 0.004*"data" + '
  '0.004*"algorithm" + 0.004*"function" + 0.004*"using" + 0.004*"one" + '
  '0.003*"number" + 0.003*"problem"'),
 (3,
  '0.005*"algorithm" + 0.005*"time" + 0.005*"data" + 0.005*"model" + '
  '0.005*"set" + 0.005*"learning" + 0.004*"log" + 0.004*"function" + '
  '0.003*"using" + 0.003*"one"'),
 (4,
  '0.006*"learning" + 0.005*"model" + 0.005*"data" + 0.004*"set" + '
  '0.004*"using" + 0.004*"algorithm" + 0.004*"one" + 0.004*"function" + '
  '0.003*"distribution" + 0.003*"time"'),
 (5,
  '0.005*"learning" + 0.005*"time" + 0.005*"model" + 0.005*"algorithm" 